In [ ]:
% matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from pipeline import Dataset
from datatools import data_quantization

In [ ]:
dataset = Dataset.load('../../data/parsed')

In [ ]:
quantized_fundamental, _ = data_quantization(dataset.fundamental.set_index(['asset', 'day']))

In [ ]:
full_df = pd.concat(
    [dataset.fundamental.set_index(['asset', 'day']), dataset.ref_return.set_index(['asset', 'day'])],
    axis=1).dropna()

In [ ]:
df = full_df.swaplevel().sort_index()
df.head()

In [ ]:
# X = df[['turnoverRatio_QUANTILE', 'transactionAmount_QUANTILE', 'pb_QUANTILE', 'ps_QUANTILE', 'pe_ttm_QUANTILE', 'pe_QUANTILE', 'pcf_QUANTILE']]
# X = df[['turnoverRatio', 'transactionAmount', 'pb', 'ps', 'pe_ttm', 'pe', 'pcf']]
# y = df['return']

original_feature = ['turnoverRatio', 'transactionAmount', 'pb', 'ps', 'pe_ttm', 'pe', 'pcf']
quantile_feature = ['turnoverRatio_QUANTILE', 'transactionAmount_QUANTILE', 'pb_QUANTILE', 'ps_QUANTILE', 'pe_ttm_QUANTILE', 'pe_QUANTILE', 'pcf_QUANTILE']

tscv = TimeSeriesSplit(n_splits=99)
test_prediction = Series(dtype=float)
test_reference = Series(dtype=float)
train_test_scores = []
for train, test in tqdm(tscv.split(df)):
    X, _ = data_quantization(df[original_feature])
    reg = LinearRegression().fit(X.iloc[train], df['return'].iloc[train])
    train_score = reg.score(X.iloc[train], df['return'].iloc[train])
    test_prediction = pd.concat([test_prediction, Series(reg.predict(X.iloc[test]), index=X.index[test])])
    test_reference = pd.concat([test_reference, df['return'].iloc[test]])
    test_score = r2_score(test_reference, test_prediction)
    # test_score = reg.score(X.iloc[test], y.iloc[test])
    train_test_scores.append((train_score, test_score))
    # print(f'train score: {train_score:.4f}, test score: {test_score:.4f}')

# final_model = LinearRegression().fit(X, y)

In [ ]:
plt.figure(1)
plt.plot([_[0] for _ in train_test_scores], label='train')
plt.plot([_[1] for _ in train_test_scores], label='test')
plt.ylim([-0.1, 0.1])
plt.legend(loc='best')

In [ ]:
pd.concat([test_reference, test_prediction], axis=1)